In [1]:
# import comet_ml at the top of your file
from comet_ml import Experiment
experiment = Experiment(
            project_name="music-genre-multiclass-classification",
            workspace="wodenwang820118",
        )
import comet_ml
import logging

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in 'c:\\D_future\\machine learning\\music_genre_classification\\github' and lookings in parents. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/6ec1e8ba95c24d79886d64697466023c



In [2]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("comet_ml")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras

%matplotlib inline
sns.set_style('whitegrid')

In [12]:
# tensorflow 2.7 
import tensorflow as tf 
from tensorflow.keras.layers import Input,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model, Sequential

In [4]:
# scale the numeric data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [5]:
df_train_features = pd.read_csv('train_features.csv')
df_train_labels = pd.read_csv('train_labels.csv')

df_valid_features = pd.read_csv('valid_features.csv')
df_valid_labels = pd.read_csv('valid_labels.csv')

df_test_features = pd.read_csv('test_features.csv')

In [6]:
num_train_data = df_train_features.iloc[:,9:]
num_valid_data = df_valid_features.iloc[:,9:]
num_test_data = df_test_features.iloc[:,9:]

In [7]:
num_train = scaler.fit_transform(num_train_data)
num_valid = scaler.fit_transform(num_valid_data)
num_test = scaler.fit_transform(num_test_data)

In [8]:
num_train_y = pd.get_dummies(df_train_labels['genre'])
num_valid_y = pd.get_dummies(df_valid_labels['genre'])

In [9]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced', np.unique(df_train_labels['genre']),df_train_labels['genre'])
class_weight = {i :class_weight[i] for i in range(8)}

C:\Users\alex0\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass classes=['classic pop and rock' 'dance and electronica' 'folk' 'jazz and blues'
 'metal' 'pop' 'punk' 'soul and reggae'], y=0                      metal
1                       folk
2                       folk
3             jazz and blues
4                       folk
                ...         
7673         soul and reggae
7674                   metal
7675    classic pop and rock
7676                   metal
7677                   metal
Name: genre, Length: 7678, dtype: object as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [10]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=5)

In [13]:
class AudioGenreClassifier:
    def __init__(self,num_train,num_train_y,num_valid,num_valid_y,class_weight,early_stop,experiment):
        self.num_train = num_train
        self.num_train_y = num_train_y
        self.num_valid = num_valid
        self.num_valid_y = num_valid_y
        self.class_weight = class_weight
        self.early_stop = early_stop
        self.experiment = experiment
    
    def build_model(self):
        # Build the model
        model = Sequential()
        model.add(
            Dense(
                self.experiment.get_parameter("first_layer_units"),
                activation='elu',
                input_shape=(num_train.shape[1],)
            )
        )
        model.add(Dropout(0.5))
        model.add(BatchNormalization())

        model.add(Dense(32,activation='elu'))
        model.add(Dropout(0.3))
        model.add(BatchNormalization())
        
        model.add(Dense(8,activation='softmax'))
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
    
    def train_model(self):
        # Train the model
        model = self.build_model()
        model.fit(
            self.num_train,
            self.num_train_y,
            batch_size=self.experiment.get_parameter("batch_size"),
            epochs=self.experiment.get_parameter("epochs"),
            validation_data=(self.num_valid,self.num_valid_y),
            class_weight=self.class_weight,
            callbacks=[self.early_stop]
        )
        return model
    
    def evaluate_model(self):
        # Evaluate the model
        model = self.train_model()
        score = model.evaluate(self.num_valid,self.num_valid_y)
        LOGGER.info(f"{ score }")
    
    def grid_search(self, config_dict):
        opt = comet_ml.Optimizer(config_dict)
        for self.experiment in opt.get_experiments(project_name="music-genre-multiclass-classification"):
            self.experiment.log_parameters("epochs", 10)

            self.build_model()
            self.train_model()
            self.evaluate_model()
            self.experiment.end()

In [14]:
audio_model = AudioGenreClassifier(num_train,num_train_y,num_valid,num_valid_y,class_weight,early_stop,experiment)
# mu is the mean number of units, sigma is the standard deviation
audio_model.grid_search({
    "algorithm": "bayes",
    "name": "Optimize Music Classification Network",
    "spec": {"maxCombo": 10, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "first_layer_units": {"type": "discrete", "values": [128, 256, 300, 400, 450, 500, 550]},
        "batch_size": {"type": "discrete", "values": [16, 32, 64]},
        "epochs": {"type": "discrete", "values": [20, 30, 40]},
    },
    "trials": 1,
})

COMET INFO: COMET_OPTIMIZER_ID=52c69d224f9644e29611232896af3da4
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 63, 'endTime': None, 'id': '52c69d224f9644e29611232896af3da4', 'lastUpdateTime': None, 'maxCombo': 10, 'name': 'Optimize Music Classification Network', 'parameters': {'batch_size': {'type': 'discrete', 'values': [16, 32, 64]}, 'epochs': {'type': 'discrete', 'values': [20, 30, 40]}, 'first_layer_units': {'type': 'discrete', 'values': [128, 256, 300, 400, 450, 500, 550]}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 10, 'metric': 'loss', 'minSampleSize': 100, 'objective': 'minimize', 'retryAssignLimit': 0, 'retryLimit': 1000}, 'startTime': 26781910596, 'state': {'mode': None, 'seed': None, 'sequence': [], 'sequence_i': 0, 'sequence_pid': None, 'sequence_retry': 0, 'sequence_retry_count': 0}, 'status': 'running', 'suggestion_count': 0, 'trials': 1, 'version': '2.0.1'}
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experimen

Epoch 1/40
120/120 [==============================] - 3s 7ms/step - loss: 1.7426 - accuracy: 0.3537 - val_loss: 1.4808 - val_accuracy: 0.4756
Epoch 2/40
120/120 [==============================] - 0s 4ms/step - loss: 1.4957 - accuracy: 0.4331 - val_loss: 1.4435 - val_accuracy: 0.4911
Epoch 3/40
120/120 [==============================] - 0s 4ms/step - loss: 1.4142 - accuracy: 0.4603 - val_loss: 1.3764 - val_accuracy: 0.5067
Epoch 4/40
120/120 [==============================] - 1s 4ms/step - loss: 1.3553 - accuracy: 0.4825 - val_loss: 1.3685 - val_accuracy: 0.4889
Epoch 5/40
120/120 [==============================] - 0s 4ms/step - loss: 1.3344 - accuracy: 0.4923 - val_loss: 1.3861 - val_accuracy: 0.5067
Epoch 6/40
120/120 [==============================] - 0s 3ms/step - loss: 1.3084 - accuracy: 0.5066 - val_loss: 1.3011 - val_accuracy: 0.5667
Epoch 7/40
120/120 [==============================] - 0s 3ms/step - loss: 1.2812 - accuracy: 0.5146 - val_loss: 1.2912 - val_accuracy: 0.5533
Epoch 

COMET INFO: [1.359973430633545, 0.5111111402511597]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/90ff3416033e4b90a008e56691e680df
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [24]                : (0.3537379503250122, 0.5363376140594482)
COMET INFO:     batch_accuracy [288]         : (0.109375, 0.609375)
COMET INFO:     batch_loss [288]             : (1.0254137516021729, 2.8981246948242188)
COMET INFO:     epoch_duration [24]          : (0.39000000000010004, 2.9529999999999745)
COMET INFO:     loss [24]                    : (1.2004743814468384, 1.7426443099975586)
COMET INFO:     val_accuracy [24]            : (0.46222221851348877, 0.5666666626930237)
COMET INFO:     val_loss [24]                : (1.2806291580200195

Epoch 1/30
240/240 [==============================] - 3s 8ms/step - loss: 1.8393 - accuracy: 0.3265 - val_loss: 1.4886 - val_accuracy: 0.4467
Epoch 2/30
240/240 [==============================] - 1s 6ms/step - loss: 1.5814 - accuracy: 0.3944 - val_loss: 1.4097 - val_accuracy: 0.4844
Epoch 3/30
240/240 [==============================] - 2s 8ms/step - loss: 1.4924 - accuracy: 0.4371 - val_loss: 1.3978 - val_accuracy: 0.5044
Epoch 4/30
240/240 [==============================] - 2s 8ms/step - loss: 1.4511 - accuracy: 0.4523 - val_loss: 1.4024 - val_accuracy: 0.4822
Epoch 5/30
240/240 [==============================] - 2s 7ms/step - loss: 1.4158 - accuracy: 0.4587 - val_loss: 1.3603 - val_accuracy: 0.5000
Epoch 6/30
240/240 [==============================] - 2s 7ms/step - loss: 1.3828 - accuracy: 0.4728 - val_loss: 1.4171 - val_accuracy: 0.4889
Epoch 7/30
240/240 [==============================] - 2s 7ms/step - loss: 1.3693 - accuracy: 0.4906 - val_loss: 1.3744 - val_accuracy: 0.4956
Epoch 

COMET INFO: [1.405808687210083, 0.4911110997200012]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/3e43309d1c85413c997dbd91c2298026
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [30]                : (0.3265173137187958, 0.5177129507064819)
COMET INFO:     batch_accuracy [720]         : (0.09375, 0.59375)
COMET INFO:     batch_loss [720]             : (1.0479601621627808, 2.922708749771118)
COMET INFO:     epoch_duration [30]          : (1.4379999999998745, 2.6569999999999254)
COMET INFO:     loss [30]                    : (1.251456618309021, 1.8392884731292725)
COMET INFO:     val_accuracy [30]            : (0.4333333373069763, 0.5288888812065125)
COMET INFO:     val_loss [30]                : (1.3071691989898682, 1.48

...

COMET INFO: [1.3526657819747925, 0.504444420337677]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/a69c83301af24af8aec8800d3298dc83
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [28]                : (0.3343318700790405, 0.5195363163948059)
COMET INFO:     batch_accuracy [1344]        : (0.11931817978620529, 0.625)
COMET INFO:     batch_loss [1344]            : (0.9241374731063843, 3.2600479125976562)
COMET INFO:     epoch_duration [28]          : (1.7029999999999745, 6.2970000000000255)
COMET INFO:     loss [28]                    : (1.2826653718948364, 1.777367353439331)
COMET INFO:     val_accuracy [28]            : (0.4533333480358124, 0.5533333420753479)
COMET INFO:     val_loss [28]                : (1.30638706684

Epoch 1/40
120/120 [==============================] - 2s 12ms/step - loss: 1.7569 - accuracy: 0.3474 - val_loss: 1.4617 - val_accuracy: 0.4444
Epoch 2/40
120/120 [==============================] - 1s 11ms/step - loss: 1.5260 - accuracy: 0.4301 - val_loss: 1.4599 - val_accuracy: 0.4622
Epoch 3/40
120/120 [==============================] - 1s 12ms/step - loss: 1.4217 - accuracy: 0.4697 - val_loss: 1.3463 - val_accuracy: 0.4911
Epoch 4/40
120/120 [==============================] - 1s 12ms/step - loss: 1.3644 - accuracy: 0.4930 - val_loss: 1.3338 - val_accuracy: 0.5378
Epoch 5/40
120/120 [==============================] - 2s 13ms/step - loss: 1.3459 - accuracy: 0.4904 - val_loss: 1.3373 - val_accuracy: 0.5067
Epoch 6/40
120/120 [==============================] - 1s 12ms/step - loss: 1.3109 - accuracy: 0.5091 - val_loss: 1.3349 - val_accuracy: 0.4978
Epoch 7/40
120/120 [==============================] - 1s 12ms/step - loss: 1.2911 - accuracy: 0.5124 - val_loss: 1.3307 - val_accuracy: 0.5289

COMET INFO: [1.3165874481201172, 0.5244444608688354]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/98db041b521d47a8a04646961a8f079b
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [44]                : (0.34735608100891113, 0.5696796178817749)
COMET INFO:     batch_accuracy [528]         : (0.125, 0.65625)
COMET INFO:     batch_loss [528]             : (0.9216297268867493, 2.485959768295288)
COMET INFO:     epoch_duration [44]          : (0.375, 2.219000000000051)
COMET INFO:     loss [44]                    : (1.136260747909546, 1.761550784111023)
COMET INFO:     val_accuracy [44]            : (0.4444444477558136, 0.5711110830307007)
COMET INFO:     val_loss [44]                : (1.2492823600769043, 1.4616572856903076)

Epoch 1/30
120/120 [==============================] - 1s 5ms/step - loss: 1.7824 - accuracy: 0.3476 - val_loss: 1.5514 - val_accuracy: 0.4067
Epoch 2/30
120/120 [==============================] - 1s 4ms/step - loss: 1.5103 - accuracy: 0.4381 - val_loss: 1.4683 - val_accuracy: 0.4600
Epoch 3/30
120/120 [==============================] - 0s 4ms/step - loss: 1.4029 - accuracy: 0.4698 - val_loss: 1.4101 - val_accuracy: 0.4778
Epoch 4/30
120/120 [==============================] - 0s 4ms/step - loss: 1.3755 - accuracy: 0.4825 - val_loss: 1.3370 - val_accuracy: 0.5178
Epoch 5/30
120/120 [==============================] - 0s 4ms/step - loss: 1.3433 - accuracy: 0.5003 - val_loss: 1.3193 - val_accuracy: 0.5133
Epoch 6/30
120/120 [==============================] - 0s 3ms/step - loss: 1.2996 - accuracy: 0.5104 - val_loss: 1.3251 - val_accuracy: 0.5244
Epoch 7/30
120/120 [==============================] - 0s 4ms/step - loss: 1.2885 - accuracy: 0.5175 - val_loss: 1.3738 - val_accuracy: 0.4844
Epoch 

COMET INFO: [1.3440489768981934, 0.5177778005599976]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/2074fccf4f4f4dd09ba2df934f667036
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [26]                : (0.34540244936943054, 0.5444126129150391)
COMET INFO:     batch_accuracy [312]         : (0.046875, 0.6875)
COMET INFO:     batch_loss [312]             : (0.8503199815750122, 3.4496145248413086)
COMET INFO:     epoch_duration [26]          : (0.375, 1.25)
COMET INFO:     loss [26]                    : (1.1834183931350708, 1.7824487686157227)
COMET INFO:     val_accuracy [26]            : (0.40666666626930237, 0.5555555820465088)
COMET INFO:     val_loss [26]                : (1.2552084922790527, 1.5513761043548584)
COMET 

In [13]:
model = Sequential()
model.add(
    Dense(
        400,
        activation='elu',
        input_shape=(num_train.shape[1],)
    )
)
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(32,activation='elu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(8,activation='softmax'))
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [14]:
model.fit(
    num_train,
    num_train_y,
    batch_size=64,
    epochs=40,
    validation_data=(num_valid,num_valid_y),
    class_weight=class_weight,
    callbacks=[early_stop]
)

Epoch 1/40
120/120 [==============================] - 2s 5ms/step - loss: 1.7834 - accuracy: 0.3635 - val_loss: 1.4567 - val_accuracy: 0.4556
Epoch 2/40
120/120 [==============================] - 0s 4ms/step - loss: 1.5334 - accuracy: 0.4332 - val_loss: 1.4492 - val_accuracy: 0.4800
Epoch 3/40
120/120 [==============================] - 1s 4ms/step - loss: 1.4282 - accuracy: 0.4693 - val_loss: 1.4213 - val_accuracy: 0.4822
Epoch 4/40
120/120 [==============================] - 1s 4ms/step - loss: 1.3810 - accuracy: 0.4857 - val_loss: 1.4317 - val_accuracy: 0.4844
Epoch 5/40
120/120 [==============================] - 0s 4ms/step - loss: 1.3466 - accuracy: 0.5072 - val_loss: 1.3601 - val_accuracy: 0.5022
Epoch 6/40
120/120 [==============================] - 1s 4ms/step - loss: 1.3099 - accuracy: 0.5104 - val_loss: 1.3466 - val_accuracy: 0.5200
Epoch 7/40
120/120 [==============================] - 1s 4ms/step - loss: 1.3052 - accuracy: 0.5125 - val_loss: 1.3575 - val_accuracy: 0.5267
Epoch 

In [25]:
from sklearn.metrics import classification_report
validation = model.predict(num_valid)
validation = validation.argmax(axis=1)
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}
predict_label_array = np.vectorize(classes_mapping.get)(validation)
correct_labels = df_valid_labels['genre'].values
print(classification_report(correct_labels,predict_label_array))

                       precision    recall  f1-score   support

 classic pop and rock       0.38      0.38      0.38        55
dance and electronica       0.44      0.44      0.44        45
                 folk       0.40      0.62      0.49        64
       jazz and blues       0.42      0.39      0.40        44
                metal       0.93      0.77      0.84        66
                  pop       0.65      0.50      0.56        74
                 punk       0.53      0.70      0.61        44
      soul and reggae       0.82      0.55      0.66        58

             accuracy                           0.55       450
            macro avg       0.57      0.55      0.55       450
         weighted avg       0.59      0.55      0.56       450



In [ ]:
model.save('audio_only_model')

INFO:tensorflow:Assets written to: audio_only_model\assets
